# Practice Session 08: Data streams

In this session we will take a large corpus of documents and compute some statistics using data streams methods.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

Author: <font color="blue">Your name here</font>

E-mail: <font color="blue">Your e-mail here</font>

Date: <font color="blue">The current date here</font>

In [ ]:
import io
import nltk
import gzip
import random
import statistics
import secrets
import re

# 0. Dataset and how to iterate

The input file contain documents in English presented at the European Parliament, collected into the [DCEP](https://ec.europa.eu/jrc/en/language-technologies/dcep) corpus. In this case, we have sampled only documents of type 'REPORT'. What we want is to compute the most frequent words in this dataset. 

During this practice, we will **never** load this file in main memory.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [ ]:
# Name of input file
INPUT_FILE = "DCEP-reports-en.txt.gz"

The function `read_by_words` is a [generator](https://wiki.python.org/moin/Generators), that is, a function that behaves as an iterator. This is a common pattern used in stream processing, and in Python is implemented with the `yield` keyword, instead of `return`.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [ ]:
# Producer in Python that reads a filename by words
def read_by_words(filename, max_words=-1, report_every=-1):
    
    # Open the input file
    with gzip.open(INPUT_FILE, "rt") as file:
        
        # Initialize counter of words to stop at max_words
        counter = 0
    
        # Regular expression to identify words having 4 letters or more and beginning with a-z
        word_expr = re.compile('^[a-z][a-z0-9]{4,}$', re.IGNORECASE)

        # Iterate through lines in the file
        for line in file:
            
            if counter > max_words and max_words != -1:
                break
                
            for word in nltk.word_tokenize(line):
                if word_expr.match(word):
                    counter += 1
                    
                    # Report
                    if (report_every != -1) and (counter % report_every == 0):
                        if max_words == -1:
                            print("- Read %d words so far" % (counter))
                        else:
                            print("- Read %d/%d words so far" % (counter, max_words))

                    # Produce the word
                    yield word

We will do a first pass over the data. Here we will read only the first 300K words. Try with a larger limit if your computer is fast, with a lower limit if your computer is slow. Find something that makes one pass take about 30 seconds and use it for development.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [ ]:
# Iterate through the file
for word in read_by_words(INPUT_FILE, max_words=300000, report_every=100000):
    # Prints 0.005% of words
    if random.random() < 0.00005:
        print("Current word '%s'" % (word)) 

# 1. Determine approximately the top-5 words

Instead of loading the entire dataset in main memory, we will use reservoir sampling to determine approximately the top-5 words.

**Reservoir sampling**: In reservoir sampling, if we have a reservoir of size S:

* We store the first S elements of the stream
* When the n<sup>th</sup> element arrives (let's call it X<sub>n</sub>):
   * With probability 1 - s/n, we ignore this element.
   * With probability s/n, we:
      * Discard a random element from the reservoir
      * Add element X<sub>n</sub> to the reservoir (calling *add_to_reservoir*)
      
<font size="-1" color="gray">(Remove this cell when delivering.)</font>

Implement a function `add_reservoir(reservoir, item, max_size)` that adds an item to the reservoir, maintaining its size. If the reservoir is already of size *max_size*, a random item is selected and evicted *before* adding the item. It is important to evict an old item *before* adding the new item. Use the following skeleton:

```python

def add_to_reservoir(reservoir, item, max_reservoir_size):
    # YOUR CODE HERE
    assert(len(reservoir) <= max_reservoir_size)
```

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code for "add_reservoir"</font>

Create a function to iterate through the file using the reservoir sampling method seen in class. In this function you will decide, for every item, whether to call *add_to_reservoir* or to ignore the item.

You can use the following skeleton:

```python
def reservoir_sampling(filename, reservoir_size, max_words=-1, report_every=-1):
    reservoir = []
    
    words_read = 0
    
    for word in read_by_words(filename, max_words=max_words, report_every=report_every):
    
            # YOUR CODE HERE

    return (words_read, reservoir)
```

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code for "reservoir_sampling"</font>

Test your function using the following code:

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [ ]:
reservoir_size = 500
(items_seen, reservoir) = reservoir_sampling(INPUT_FILE, reservoir_size, max_words=300000, report_every=100000)

print("Number of items seen    : %d" % items_seen)
print("Number of items sampled : %d" % len(reservoir) )

The reservoir contains repeated items. You can compute the absolute frequencies of the top 5 using the following code.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [ ]:
freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:10]
for absolute_frequency, word in most_frequent_items:
    print("%d %s" % (absolute_frequency, word))

Write code to compute the 5 most frequent items in the reservoir and their relative frequencies, as percentages.

If you see an item C times in the reservoir, you can estimate its absolute frequency as *C x dataset_size / reservoir_size*, i.e., the number of times it appears in the entire dataset (*dataset_size* is the total number of items read). 

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to print the top items and their relative and absolute frequencies</font>

For various sizes of the reservoir, e.g., 50, 100, 500, ..., list the top-5 words and your estimate of their absolute and relative frequency in the entire dataset.
 
<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Increase the max limit of words so that one pass takes no more than 5 minutes to be completed. Replace this cell with your code to try different reservoir sizes. In each case, print your estimate for the relative and absolute frequency of the words in the entire dataset.</font>

Find by trial and error, and include in your report, the minimum reservoir size you need to have somewhat stable results (e.g., the same top-3 words in two consecutive runs of the algorithm).

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Remove the max limit of words and re-run. Replace this cell with a brief commentary indicating what reservoir size you would recommend to use, and your overall conclusions.</font>

# 2. Determine approximately the distinct number of words

We will estimate the number of distinct words without creating a dictionary or hash table, but instead, we will use the Flajolet-Martin probabilistic counting method.

**Flajolet-Martin probabilistic counting**:

* For several passes
   * Create hash funcion h
   * For every element *u* in the stream:
      * Compute hash value *h(u)*
      * Let *r(u)* be the number of trailing zeroes in *h(u)*
      * Maintain *R* as the maximum value of *r(u)* seen so far
   * Add *2<sup>R</sup>* as an estimate for the number of distinct elements *u* seen
* The final estimate is the average or the median of the estimates found in each pass

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

Use this function to count trailing zeroes in the binary representation of a number.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [1]:
def count_trailing_zeroes(number):
    count = 0
    while number & 1 == 0:
        count += 1
        number = number >> 1
    return count

Use this function to generate a random hash function. Note this generates a function, so you can do `hash_function = random_hash_function()` and then call `hash_function(x)` to compute the hash value of `x`.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [2]:
def random_hash_function():
    # We use a cryptographically safe generator for the salt of our hash function
    salt = secrets.token_bytes(32)
    return lambda string: hash(string + str(salt))

Perform *number_of_passes* passes over the file, reading the entire file on each pass (we don't use the reservoir in this part). In each pass, create a new hash function and use it to hash userids. Keep the maximum number of trailing zeroes seen in the hash value of a userid. 

```python
number_of_passes = 10
estimates = []

for i in range(number_of_passes):
    # YOUR_CODE_HERE: read the file and generate an estimate
    
    estimates.append(estimate)
    print("Estimate on pass %d: %d distinct words" % (i+1, estimate))
```

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to perform the requested number of passes.</font>

The next cell should print the median of the estimates.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [ ]:
print("* Average of estimates: %.1f" % statistics.mean(estimates))
print("* Median  of estimates: %.1f" % statistics.median(estimates))

Now, double the number of passes (20 instead of 10) and re-run. Print the median of your estimates.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to compute the average and the median of 20 passes.</font>

Finally, remove the limit of max words that we used when reading the file, and run the algorithm for 10 passes.

If you notice the algorithm will take more than one hour to run, you can put back the limit of max words and increase it until the total running time of the 10 passes is about one hour. In other words, we are not asking you to run this for more than one hour.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with the limit of max words that you used (or "no max words limit" if you removed the max words limit), the time it took to run the algorithm, the estimate you obtained in each of the 10 passes, the average estimate, and the median estimate.</font>

<font size="+1" color="red">Replace this cell with your commentary indicating whether the average or the median seems to be a better estimator in this case.</font>

# DELIVER (individually)

Remember to read the section on "delivering your code" in the [course evaluation guidelines](https://github.com/chatox/data-mining-course/blob/master/upf/upf-evaluation.md).

Deliver a zip file containing:

* This notebook

## Extra points available

For more learning and extra points, notice that the number of **distinct** words in a corpus, as a function of the **total** number of words in the corpus, follows an empirical law known as [Heap's Law](https://en.wikipedia.org/wiki/Heaps%27_law). Repeat the probabilistic counting experiment for various values of `max_word` and plot the total number of words read versus the number of distinct words (remember to label axes). Check if it follows Heap's law. Please note that using probabilistic counting means a substantial amount of noise will be introduced and perhaps the Heap's law will not be clear in your plot.

**Note:** if you go for the extra points, add ``<font size="+2" color="blue">Additional results: Heap's law</font>`` at the top of your notebook. 

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>